# Final Project

In [30]:
# Import statements
import os
import re
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
def create_df(c):
    df = pd.DataFrame()
    divider = []
    names = []
    for f in os.listdir("tweets/%s" % c):
        try:
            dft = pd.read_csv("tweets/%s/%s" % (c, f))
            if len(divider) == 0:
                divider.append(len(dft) - 1)
            else:
                divider.append(len(dft) + divider[-1])
            df = pd.concat([df, dft])
            # parse output_name.csv string
            names.append(f[7:][:-4])
        except:
            print(f)
    return df, divider, names

In [32]:
categories = ["fast food", "airlines", "leagues", "colleges", "streaming platforms", "news", "tech giants", "celebrities"]

fast_food, ff_divider, ff_names = create_df(categories[0])
leagues, lg_divider, lg_names = create_df(categories[2])
news, nw_divider, nw_names = create_df(categories[5])
celebs, cb_divider, cb_names = create_df(categories[7])



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [33]:
def clean_tweets(tweets):
    categories = len(tweets)
    print(categories)
    all_tweets = []
    for c in range(categories):
        t_new = []
        count = 0
        for t in tweets[c]:
            try:
                w = t.split()
                w = get_clean_words(w)
                tc = " ".join(w)
                t_new.append(tc)
            except:
                count += 1
        print(count, len(t_new))
        all_tweets.append(t_new)
    return all_tweets

In [34]:
%matplotlib inline

import numpy as np
import random
import sys
import io
import requests as rq
from bs4 import BeautifulSoup
import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from datetime import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [35]:
celebs_list = list(celebs['text'])
print(len(celebs_list))
print(cb_divider)
text = ""
curr = 0
for i in range(10):
    print(curr)
    for j in range(1000):
        text += " " + str(celebs_list[curr + j])
    curr = cb_divider[i]
print(text[:500])

65354
[8916, 18247, 18247, 22131, 24985, 29830, 39830, 41023, 51023, 61023, 65353]
0
8916
18247
18247
22131
24985
29830
39830
41023
51023
 Love you I grew up in a cogic family church in Detroit.... music was my vehicle to a higher power... I like to catch the spirit on stage, I don’t know no other way  Period. The PEOPLE are making these calls... not the label. I can’t wait for y’all to discover Lingerie  I LOVE SOULMATE! That was supposed to be the 2nd single believe it or not.... but Good As Hell started climbing the streaming charts Guess we gotta wait for the world to fall in love with it  Y’all think it was the plan for all m


In [36]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Epoch 1/50
 9472/23638 [===========>..................] - ETA: 38s - loss: 3.4104 - categorical_crossentropy: 3.4104 - accuracy: 0.1754

KeyboardInterrupt: 

In [19]:
fast_food_list = list(fast_food['text'])
print(len(fast_food_list))
print(ff_divider)
text = ""
curr = 0
for i in range(10):
    for j in range(1000):
        text += " " + str(fast_food_list[curr + j])
    curr = ff_divider[i]
print(text[:500])

100000
[9999, 19999, 29999, 39999, 49999, 59999, 69999, 79999, 89999, 99999]
 these parties are getting wild, be careful out there tonight! Meet the Freaky Fast® Order button. Life just got a whole lot easier. I'm really sorry for the hold up today! Give the store a call and see what's up! I LOVE YOUR LOVE! nan I'm sorry! Let the store know right away so they can make it right! So glad you could get in on the sample sub love!! nan Raee  nan Any takers? nan I'm sorry for the wait last night, slow is not how I roll! Can you please DM me the store you ordered from along wit


In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Epoch 1/50
23638/23638 [==============================] - 55s 2ms/step - loss: 2.8180 - categorical_crossentropy: 2.8180 - accuracy: 0.2764

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "manII), and stand up from @jimmycarr! #F"
manII), and stand up from @jimmycarr! #FallonTonight Thank you have a sour and and and the prout and and and and and a all the bean in the make the promer and the soooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
----- diversity: 0.5
----- Generating with seed: "manII), and stand up from @jimmycarr! #F"
manII), and stand up from @jimmycarr! #FallonTonight I wall and all we all arooola be for the - @Nandaght #FallonTonight @Theanter #FallonTonight #FellonTonight Fanting the a or the slone the a dan

itter_share&amp;igshid=vedycrc8tray Sature and the movie in the first this brother to this said with @verethallore Jan it. The crew to one home of the show is the works and the best time to see the life my got and canners of the rome of my day. I love all out the could done and the funny our that mean from @TWEThe out to @RealParting @NBCWorldofDance Thank you. The best to the brother I don’t know the could stather what the honered to t
----- diversity: 1.0
----- Generating with seed: "itter_share&amp;igshid=vedycrc8tray Satu"
itter_share&amp;igshid=vedycrc8tray Sature for get taghing.... BuckToreransing and saye like, ansone didfurtanden soller grate day? Thank you comi sibustly love you! Plus, a performance hage nan it to mixed to br and trars josh apeter show!! I'm week tode in a touching & back tosem this dream so much ! sercovier. to do making the @nolmmo @deaspio, It's would gottrial really Never here: Go at being Pick. My today on Fire Decfult so much.
Epoch 6/50
23638/23638 [==

23638/23638 [==============================] - 79s 3ms/step - loss: 1.6237 - categorical_crossentropy: 1.6237 - accuracy: 0.5497

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "ert Pattinson #hustlersmovie : Art Strei"
ert Pattinson #hustlersmovie : Art Streinth #TheUpside #TheUpside #TheUpside #HustleHart https://www.instagram.com/p/BxTshcsFFkZ/?utm_source=ig_twitter_share&amp;igshid=1wwwwwwwwttt Lead. Thank you for the best finally and the show to the best that you are you are with @TheMander with @NBCWorldofDance @NBCWorldofDance #FallonTonight Tonight! @NBCWorldofDance @NBCWorldofDance @NBCWorldofDance @NBCWorldofDance @NBCWorldofDance @NBCWorldof
----- diversity: 0.5
----- Generating with seed: "ert Pattinson #hustlersmovie : Art Strei"
ert Pattinson #hustlersmovie : Art Strein Make hull and the best of the most come the first that you’ll be on the show! My partners in the show tonight! @KevinHart4real and @PostMalone stops by, and @theroonb

In [21]:
leagues_list = list(leagues['text'])
print(len(leagues_list))
print(lg_divider)
text = ""
curr = 0
for i in range(10):
    for j in range(1000):
        text += " " + str(leagues_list[curr + j])
    curr = lg_divider[i]
print(text[:500])

91446
[9999, 19999, 29999, 39999, 49999, 54847, 64847, 74847, 84847, 91445]
 We're especially thankful for baseball. #Thanksgiving The hot corner is STACKED. Who is your All-MLB pick? http://mlb.com/allmlb Top of their class. Vote All-MLB: http://mlb.com/allmlb Don’t stop short in telling us your choice. Vote for the All-MLB Team: http://mlb.com/allmlb .@Pirates officially name Derek Shelton as manager. Shelton spent the past 2 seasons as the Twins bench coach. After a stellar second half in relief, LHP Drew Pomeranz reportedly agrees to deal with Padres, per @Feinsand.


In [22]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars

Epoch 1/50
25469/25469 [==============================] - 50s 2ms/step - loss: 2.8890 - categorical_crossentropy: 2.8890 - accuracy: 0.2748

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "on the Superdome turf. @run__cmc @idjmoo"
on the Superdome turf. @run__cmc @idjmook and win the @Walllay 20 PTS Watch from the @NBAAlick Sports app Watch from @Alling #WNBAFinals pres the @NBAChamp Watch from @Allind win the @NFL and the @NBAFinals and wink the @Wallon Match from the @Auddors and the @NBAChamp Watch for the @NFL and #WNBAFinals and the @NBAFinals of @Supion The @NBAFinals of the @NBAAnday and the @Sunders Con @NFL and the @NBAAllayon the @Chinding the @NBAChamp
----- diversity: 0.5
----- Generating with seed: "on the Superdome turf. @run__cmc @idjmoo"
on the Superdome turf. @run__cmc @idjmooks (@NFL an 2019 #Brallight #NFL100 AFC On @SPL Champions All and Phis and #WNBAChamp Start #WNBAFinal 15 13.3: VIPG  and will to and wind at @NBAThomp Bours

D and #CMR in the #FIFAWWC: Estelle Johning and @SacramentoProud Brighton of the back the post the @PGAChampionship at https://p.ga/2019/18 @Mighannith) for the @ConnecticutSun and week in the season, continte than takes the fall has more of the @Audi #MLSCupPlayoffs .@PatrickMarior and @SaperdBowing a presented a defend the @TakerCup caneal has for @ellectinn) @NFLFantasy Top on the best the @Nationals in the #WNBAFinals .@SoundersFC a
----- diversity: 1.0
----- Generating with seed: "D and #CMR in the #FIFAWWC: Estelle John"
D and #CMR in the #FIFAWWC: Estelle Johnst best known martal studey peaces engcore with the net tram sannid't for on the roleademloher trking its ansinaleed : 15 PTS, 6 REB, 5 AST, 5 3PM Sun Carly at teamurned OT!? inals geately the sicthing spotly me histordey and goull with youtro's Befordgaals as Match Mastball #MineMBordontover three. #PLAChampionshiply pant finalowball season to berted Thisespical Indialleader meme." Leana in the w
Epoch 6/50
25469/25469 [==

25469/25469 [==============================] - 43s 2ms/step - loss: 1.5001 - categorical_crossentropy: 1.5001 - accuracy: 0.5932

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: " on the floor @ConnecticutSun 51 - @Wash"
 on the floor @ConnecticutSun 51 - @WashMystics and @SoundersFC as the @Nationals are off to the season in the @StanfordWBB season for the @ConnecticutSun the @ConnecticutSun the #WNBAFinals The @Audi #MLSCupPlayoffs The @Audi #MLSCupPlayoffs Who was all the @Nationals are to the @WashMystics and @Nationals are how to secure the @StanfordWBB season in the @ConnecticutSun the @Audi #MLSCupPlayoffs Expares to see in the #PL matches to the
----- diversity: 0.5
----- Generating with seed: " on the floor @ConnecticutSun 51 - @Wash"
 on the floor @ConnecticutSun 51 - @WashMystics Annise of @ashneway Since comminity and equalist round in the stars after on the #WNBAFinals The @Colts is to the #WNBAFinals The @StanfordWSon took and help on

ersFC! FT: @realsaltlake are headed to the #WNBAFinals prepared by @SenthonFBowls #WNBAFinals | #NHLScores BR standsup has to head someth standers at higher and the @Lakers with the @ChennaiOnKe She Seattle win the @USMSonther since is the #PGAProChamp The @SandeShos) Anniber and season in the @ConnecticutSun at the 2019 #PL matches this seeson!  #PL goals and help continues the first final the Premier Matchday Live http://bit.ly/ipl201
----- diversity: 1.0
----- Generating with seed: "ersFC! FT: @realsaltlake are headed to t"
ersFC! FT: @realsaltlake are headed to the are turks the @Lakandon17 next-we. Theyeshpetive can by Yero our fine at #PL golf-lous now all season! GOAL Sethe Karli, Austrul Forton https://on.ncaa.com/2OONByGA #PGAChamp Play to the Gatere: 21 #MLSCup! #HereWeGo : #BHIvsLARH Masolli Mulanunds. : the Game 5 of the Year-high Earter 242-pat, advances bully atting this season. #ATLvSOH // Audi #MLSCupPlayoffs 2019 Whothassein Stabliez Vilite s
Epoch 15/50
25469/25469 [=

25469/25469 [==============================] - 42s 2ms/step - loss: 1.4265 - categorical_crossentropy: 1.4265 - accuracy: 0.6107

----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: "gram. Story http://p.ga/PGA-CPP19 Reply "
gram. Story http://p.ga/PGA-CPP19 Reply to the #WNBA matches and the @ConnecticutSun for the @ChelseaFC for the @ConnecticutSun for the @Nationals are have been the @ConnecticutSun and @NCAADII matches to the 2019 #WNBA Championship become the @ConnecticutSun for the @ConnecticutSun for the @ConnecticutSun and @NCAADII matches to the #WNBA season and the @ConnecticutSun for the @ConnecticutSun and @NCAADII matches to the #WNBA came the 
----- diversity: 0.5
----- Generating with seed: "gram. Story http://p.ga/PGA-CPP19 Reply "
gram. Story http://p.ga/PGA-CPP19 Reply 6 AST to a tonight in the final start it and start to we action! #WorldSeries The USC see on mobile: http://on.nfl.com/lJfqDj .@Deymrophomis ( @realsalea Harden (@anni

...

25469/25469 [==============================] - 58s 2ms/step - loss: 1.3789 - categorical_crossentropy: 1.3789 - accuracy: 0.6217

----- Generating text after Epoch: 36
----- diversity: 0.2
----- Generating with seed: "r: Manager of the Season: Manager of the"
r: Manager of the Season: Manager of the 2019 #WNBAFinals season in the 2019 #WNBAFinals show the @Lakers on the first #WNBAFinals show in the @PatrickMahomes and the @SoundersFC at the @NationalPro #WNBAFinals .@TorontoFC show in the @PGAVillage. #PGAChamp   The @Colts in the first players the @LAClippers college basketball has been the @LAClippers to the @Colts in the @NYRangers are the #PL matches and the @WashMystics and the @PGAVil
----- diversity: 0.5
----- Generating with seed: "r: Manager of the Season: Manager of the"
r: Manager of the Season: Manager of the 2019 #WNBAFinals .@De11eDonne career-high 56 PTS and the @WashMystics be in the post-and shots her the @ConnecticutSun down the 2019 #WNBA players to she national cha

 the #WNBA is all about #WNBAPlayoffs “We win in a season-high 15 PTS, 14 REB The @WashMystics half of the @WashMystics her first Elite Career. On the booking in the #PL college season &amp; @Matt_chan1ettes has a long in the @katiannes and @ATT Call of the Monday are a @Buccaneers TD for the @AtlantaFalcons pass of the leaderboard and the @Nationals are with a teammate has your success to battle with the presented by @ATT Canada, 107/4
----- diversity: 1.0
----- Generating with seed: " the #WNBA is all about #WNBAPlayoffs “W"
 the #WNBA is all about #WNBAPlayoffs “We an include in poenty in the @49easove Week 12 COUWTO The @dallascowboys she national title: http://on.ncaa.com/2t5wsN2 #SNOvVEL // Audi #MLSCupPlayoffs HL-HAYCER. Stanfor lead PGA Leicent We Match 2. 3.5: S Pandiefleen Schusting 2020... #mersENT signers on a 7-4. Linder? "We’re behway noth ensures two accedent zone. x #MLSeniorNa. Dippenters run, Rail Yebrige to who'l Wasplace Cole to theret. 20
Epoch 42/50
25469/25469 [=

25469/25469 [==============================] - 65s 3ms/step - loss: 1.3673 - categorical_crossentropy: 1.3673 - accuracy: 0.6240

----- Generating text after Epoch: 45
----- diversity: 0.2
----- Generating with seed: "is BIG in the Conference Finals of the @"
is BIG in the Conference Finals of the @LAClippers for @NCAADIII women's all the @LAClippers and the @NCAADIII women's basketball and a career his family, friends and has a second win and a chance to the 2019 #PGAChamp The @PelicansNBA for @NCAADIII men's double for the @PGAChampionship and @NCAADIII women's back to the #NFL100 All-Time Team on @NBATV : https://p.ga/pgated and the @LAClippers to the start the @NCAADIII women’s golf and 
----- diversity: 0.5
----- Generating with seed: "is BIG in the Conference Finals of the @"
is BIG in the Conference Finals of the @LAClippers to the start and @EASPORTSNHL OT win discusses a rushing the pass to find the @LAClippers Newcast Connor McDavid (@cmcdavid97) is the following in an import

dies! #PGAChamp Matt Wallace jumps into the ball the golf college soor the decision of the @Patriots Live points at the @ConnecticutSun and Game 2 of the #WNBAFinals comes to the NCAA Watch Live: https://nba.app.link/nbaleaguepass2  @MattDoyle76 single to help named the @USAc (13-5) here in the #PGAAssistant the @NHLBlack. #FearTheDeer Harter 2 of the #WNBA leaders home of @PGAVillage of the life. Bolo retle starts win in Velocity in th
----- diversity: 1.0
----- Generating with seed: "dies! #PGAChamp Matt Wallace jumps into "
dies! #PGAChamp Matt Wallace jumps into him think the @PGASeThHowars #MLSCup The 14th post Norkics Got RUX CERE. Experion for a drama on http://folth) of by -One picks http://preml.ge/gQjJiy @arfinaUronso5. #Frok The packs opportunities against the get a San Booker's hosts with @ConnecticutSun TU) Sidebury http://bit.ly/wipl2019-4 #SNOvVEL #IPL #WIPL FIsTE. Final in Week 12!  Wolves flut @KrisesFC. Where being here. .@waymastacis17 from


Reusing TensorBoard on port 6006 (pid 33801), started 6:05:28 ago. (Use '!kill 33801' to kill it.)

In [ ]:
news_list = list(news['text'])
print(len(news_list))
text = ""
for i in range(10):
    for j in range(1000):
        text += " " + str(news_list[i*10000 + j])
print(text[:500])

In [ ]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# A1: the purpose of this block is to store the indices at which each character can be found and to store the characters
# at each index so that we can use this as input data for our generative text program
# char_indices is used when we want to determine at which index to place a particular character
# indices_char is used when we want to determine which character to put at a particular index
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# A2: the purpose of this block is to break up our sentence generation work into bite-size chunks
# the code is saying that we will only generate 40 characters at a time
# seqlen tells us how many characters  to we want total and step tells us how many characters each sequence
# we generate will have while step helps us move through each part of the text in order to generate new sequences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# A3: This block vectorizes the input and output data because the network does not work by inputting actual text.
# Therefore, we must represent each character in each sequence as a one-hot vector of boolean values. We can
# picture this as a three-dimensional block, or we can think about the x and y indexing as 
# first: sentence index,
# second: letter index within the sequence
# third: index in character vec of bools (whether or not the current character is a "a", "b", "c", ..., "space", etc.)
# the dimensionality of these vectors x and y is therefore 3
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# A4: the return sequences argument lets us allow the generated text to be outputted when we run the model so
# that we can see how our network is performing. If we flip it to False, we actually get a dimension mismatch error
# because we get all the input data instead of input data that is split up, sentence-by-sentence
# the dense layer at the end is essential to make the input data comply with the shape that is expected
# without it, the shape is (40, 83), but we need a shape of (40, 128), so it adds a layer of neurons which fine-tunes each 
# character prediction
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    # A5: diversity helps the network hesitantly generate unique output. 0.2 is not much diversity, so if the network says "Well, look here", 
    # the network might generate text that looks something like "Well, looke her", and then on the next iteration with diversity = 0.5,
    # the network generates text that diverges from the input data a little more, and then when we get to 1.0, we fully diverge from the
    # input text but the hope is that the style is still derived from that of Pulp Fiction
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

%tensorboard --logdir logs/scalars